# Crypto Performance Indicators: Omega, Sharpe, and Sortino Ratios

This guide explains how to calculate essential risk-adjusted performance metrics for cryptocurrency investments: Omega, Sharpe, and Sortino ratios. These indicators help investors make more informed decisions by quantifying the relationship between returns and risk.

## Table of Contents

- Introduction to Risk-Adjusted Performance Metrics
- Required Libraries
- Main Function Implementation
- Understanding the Indicators
    - Omega Ratio
    - Sharpe Ratio
    - Sortino Ratio
- Usage Examples
- Troubleshooting
- Interpreting Results
- Advanced Considerations

## Introduction to Risk-Adjusted Performance Metrics


When evaluating investment opportunities in cryptocurrencies, raw returns don't tell the complete story. Risk-adjusted performance metrics help investors understand the return potential relative to the risk taken. The three key metrics we'll focus on are:
1. **Omega Ratio:** Measures the probability-weighted ratio of gains versus losses for returns above and below a threshold.
2. **Sharpe Ratio:** Measures excess return per unit of total risk (standard deviation).
3. **Sortino Ratio:** Similar to Sharpe but focuses only on downside risk.

### Required Libraries

In [1]:
%pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install numpy
!pip install pandas
!pip install datetime
!pip install yfinance
!pip install ccxt
!pip install requests
!pip install time

ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import ccxt  # Alternative cryptocurrency data source
import requests  # For API calls
import time  # For rate limiting

### Main Function Implementation

In [4]:
def get_data_yfinance(ticker, start_date, end_date):
    """
    Get daily returns data from Yahoo Finance.
    
    Parameters:
    -----------
    ticker : str
        Cryptocurrency ticker symbol
    start_date : datetime
        Start date for historical data
    end_date : datetime
        End date for historical data
        
    Returns:
    --------
    pandas.Series
        Series of daily returns
    """
    # Add -USD suffix if not already present
    if not ticker.endswith('-USD'):
        ticker_yf = f"{ticker}-USD"
    else:
        ticker_yf = ticker
        
    # Download historical data
    data = yf.download(ticker_yf, start=start_date, end=end_date, progress=False)
    
    if len(data) < 2:  # Skip if not enough data
        return None
    
    # Try to use Close if Adj Close is unavailable
    if 'Adj Close' in data.columns:
        price_col = 'Adj Close'
    elif 'Close' in data.columns:
        price_col = 'Close'
    else:
        raise ValueError("Neither 'Adj Close' nor 'Close' columns available")
    
    # Calculate daily returns
    daily_returns = data[price_col].pct_change().dropna()
    
    return daily_returns

In [5]:
def get_data_ccxt(ticker, start_date, end_date):
    """
    Get daily returns data from CCXT (cryptocurrency exchange library).
    
    Parameters:
    -----------
    ticker : str
        Cryptocurrency ticker symbol
    start_date : datetime
        Start date for historical data
    end_date : datetime
        End date for historical data
        
    Returns:
    --------
    pandas.Series
        Series of daily returns
    """
    try:
        # Initialize exchange (using Binance as default)
        exchange = ccxt.binance({
            'enableRateLimit': True,  # Required by the CCXT library
        })
        
        # Convert ticker to USDT format
        symbol = f"{ticker}/USDT"
        
        # Convert datetime to milliseconds timestamp
        since = int(start_date.timestamp() * 1000)
        end_timestamp = int(end_date.timestamp() * 1000)
        
        # Get daily OHLCV data
        ohlcv = []
        while since < end_timestamp:
            data_chunk = exchange.fetch_ohlcv(symbol, '1d', since)
            if not data_chunk:
                break
            
            ohlcv.extend(data_chunk)
            since = data_chunk[-1][0] + 1  # Next timestamp after the last received
            time.sleep(exchange.rateLimit / 1000)  # Respect rate limit
            
            # Stop if we've reached or passed the end date
            if since >= end_timestamp:
                break
        
        if not ohlcv:
            return None
            
        # Convert to DataFrame
        df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.set_index('timestamp', inplace=True)
        
        # Calculate daily returns
        daily_returns = df['close'].pct_change().dropna()
        
        return daily_returns
        
    except Exception as e:
        print(f"CCXT error for {ticker}: {e}")
        return None

In [6]:
def get_data_coingecko(ticker, start_date, end_date):
    """
    Get daily returns data from CoinGecko API.
    
    Parameters:
    -----------
    ticker : str
        Cryptocurrency ticker symbol
    start_date : datetime
        Start date for historical data
    end_date : datetime
        End date for historical data
        
    Returns:
    --------
    pandas.Series
        Series of daily returns
    """
    try:
        # CoinGecko uses different IDs than ticker symbols
        # This is a simplified mapping, in practice you might need a more comprehensive solution
        ticker_mapping = {
            'BTC': 'bitcoin',
            'ETH': 'ethereum',
            'SOL': 'solana',
            'BNB': 'binancecoin',
            'XRP': 'ripple',
            'DOGE': 'dogecoin',
            'ADA': 'cardano',
            'AVAX': 'avalanche-2',
            'SHIB': 'shiba-inu',
            'DOT': 'polkadot',
            'LINK': 'chainlink',
            'TON': 'the-open-network',
            'TRX': 'tron',
            'MATIC': 'matic-network',
            'UNI': 'uniswap',
            'NEAR': 'near',
            'ICP': 'internet-computer',
            'BCH': 'bitcoin-cash',
            'XLM': 'stellar',
            'APT': 'aptos'
        }
        
        coin_id = ticker_mapping.get(ticker, ticker.lower())
        
        # Convert dates to unix timestamps (seconds)
        from_timestamp = int(start_date.timestamp())
        to_timestamp = int(end_date.timestamp())
        
        # Construct API URL
        url = f"https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart/range"
        params = {
            'vs_currency': 'usd',
            'from': from_timestamp,
            'to': to_timestamp
        }
        
        # Make API request
        response = requests.get(url, params=params)
        data = response.json()
        
        if 'prices' not in data or not data['prices']:
            return None
            
        # Convert to DataFrame
        prices_df = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
        prices_df['timestamp'] = pd.to_datetime(prices_df['timestamp'], unit='ms')
        prices_df.set_index('timestamp', inplace=True)
        
        # Resample to daily frequency (in case there are multiple data points per day)
        daily_prices = prices_df.resample('D').last()
        
        # Calculate daily returns
        daily_returns = daily_prices['price'].pct_change().dropna()
        
        return daily_returns
        
    except Exception as e:
        print(f"CoinGecko API error for {ticker}: {e}")
        return None
    
    return results

In [7]:
def get_indicator(what="omega", tickers=None, timeframes=None, risk_free_rate=0.0001, threshold=0, data_source="ccxt"):
    """
    Calculate performance indicators (Omega, Sharpe, Sortino) for a list of cryptocurrencies
    across different timeframes.
    
    Parameters:
    -----------
    what : str
        The indicator to calculate: "omega", "sharpe", or "sortino"
    tickers : list
        List of cryptocurrency tickers to analyze
    timeframes : list
        List of timeframes in days to analyze
    risk_free_rate : float
        Annual risk-free rate (default: 0.02 or 2%)
    threshold : float
        Minimum acceptable return for Omega ratio (default: 0)
    data_source : str
        Source for price data: "ccxt", "yfinance", or "coingecko" (default: "ccxt")
        
    Returns:
    --------
    DataFrame
        A DataFrame with tickers as rows, timeframes as columns, and the requested indicator as values
    """
    # Default values if not provided
    if tickers is None:
        # Top 20 cryptocurrencies by market cap (as of May 2025)
        tickers = [
            "BTC", "ETH", "SOL", "BNB", "XRP",
            "DOGE", "ADA", "AVAX", "SHIB", "DOT",
            "LINK", "TON", "TRX", "MATIC", "UNI",
            "NEAR", "ICP", "BCH", "XLM", "APT"
        ]
    
    if timeframes is None:
        timeframes = [2000, 1000, 365, 180, 90, 30, 15, 7]
    
    # Convert annual risk-free rate to daily
    daily_risk_free = (1 + risk_free_rate) ** (1/365) - 1
    
    # Create empty DataFrame to store results
    results = pd.DataFrame(index=tickers, columns=timeframes)
    
    # Get current date
    end_date = datetime.now()
    
    # Calculate indicators for each ticker and timeframe
    for ticker in tickers:
        for timeframe in timeframes:
            # Calculate start date
            start_date = end_date - timedelta(days=timeframe)
            
            try:
                # Get historical data based on selected source
                if data_source == "yfinance":
                    daily_returns = get_data_yfinance(ticker, start_date, end_date)
                elif data_source == "ccxt":
                    daily_returns = get_data_ccxt(ticker, start_date, end_date)
                elif data_source == "coingecko":
                    daily_returns = get_data_coingecko(ticker, start_date, end_date)
                else:
                    raise ValueError(f"Invalid data source: {data_source}")
                
                if daily_returns is None or len(daily_returns) < 2:
                    results.loc[ticker, timeframe] = np.nan
                    continue
                
                if what == "omega":
                    results.loc[ticker, timeframe] = calculate_omega_ratio(daily_returns, threshold)
                elif what == "sharpe":
                    results.loc[ticker, timeframe] = calculate_sharpe_ratio(daily_returns, daily_risk_free)
                elif what == "sortino":
                    results.loc[ticker, timeframe] = calculate_sortino_ratio(daily_returns, daily_risk_free)
                else:
                    raise ValueError(f"Invalid indicator: {what}. Choose 'omega', 'sharpe', or 'sortino'.")
                    
            except Exception as e:
                print(f"Error processing {ticker} for {timeframe} days: {e}")
                results.loc[ticker, timeframe] = np.nan
    
    return results

In [8]:
def get_indicator(what="omega", tickers=None, timeframes=None, risk_free_rate=0.001, threshold=0, data_source="ccxt"):  ## Corrected
    """
    Calculate performance indicators (Omega, Sharpe, Sortino) for a list of cryptocurrencies
    across different timeframes.
    
    Parameters:
    -----------
    what : str
        The indicator to calculate: "omega", "sharpe", or "sortino"
    tickers : list
        List of cryptocurrency tickers to analyze
    timeframes : list
        List of timeframes in days to analyze
    risk_free_rate : float
        Annual risk-free rate (default: 0.0001 or 0.01%)
    threshold : float
        Minimum acceptable return for Omega ratio (default: 0)
    data_source : str
        Source for price data: "ccxt", "yfinance", or "coingecko" (default: "ccxt")
        
    Returns:
    --------
    DataFrame
        A DataFrame with tickers as rows, timeframes as columns, and the requested indicator as values
    """
    # Default values if not provided
    if tickers is None:
        # Top 20 cryptocurrencies by market cap (as of May 2025)
        tickers = [
            "BTC", "ETH", "SOL", "BNB", "XRP",
            "DOGE", "ADA", "AVAX", "SHIB", "DOT",
            "LINK", "TON", "TRX", "MATIC", "UNI",
            "NEAR", "ICP", "BCH", "XLM", "APT"
        ]
    
    if timeframes is None:
        timeframes = [2000, 1000, 365, 180, 90, 30, 15, 7]
    
    # Sort timeframes in descending order to ensure we fetch the maximum timeframe first
    sorted_timeframes = sorted(timeframes, reverse=True)
    max_timeframe = sorted_timeframes[0]
    
    # Convert annual risk-free rate to daily
    daily_risk_free = (1 + risk_free_rate) ** (1/365) - 1
    
    # Create empty DataFrame to store results
    results = pd.DataFrame(index=tickers, columns=timeframes)
    
    # Get current date
    end_date = datetime.now()
    
    # Calculate indicators for each ticker
    for ticker in tickers:
        # Calculate start date for maximum timeframe
        max_start_date = end_date - timedelta(days=max_timeframe)
        
        try:
            # Get historical data based on selected source (only once per ticker)
            if data_source == "yfinance":
                full_returns_data = get_data_yfinance(ticker, max_start_date, end_date)
            elif data_source == "ccxt":
                full_returns_data = get_data_ccxt(ticker, max_start_date, end_date)
            elif data_source == "coingecko":
                full_returns_data = get_data_coingecko(ticker, max_start_date, end_date)
            else:
                raise ValueError(f"Invalid data source: {data_source}")
            
            if full_returns_data is None or len(full_returns_data) < 2:
                # If we can't get data for the maximum timeframe, mark all timeframes as NaN
                for timeframe in timeframes:
                    results.loc[ticker, timeframe] = np.nan
                continue
            
            # For each timeframe, slice the data and calculate the indicator
            for timeframe in timeframes:
                try:
                    # Calculate the start date for this timeframe
                    timeframe_start_date = end_date - timedelta(days=timeframe)
                    
                    # Create a mask for the timeframe period
                    # Use '>=' instead of '>' to include the start date
                    mask = full_returns_data.index >= timeframe_start_date
                    
                    # Slice the returns data for the current timeframe
                    timeframe_returns = full_returns_data[mask]
                    
                    # Skip if we don't have enough data points for this timeframe
                    if len(timeframe_returns) < 2:
                        results.loc[ticker, timeframe] = np.nan
                        continue
                    
                    # Calculate the requested indicator
                    if what == "sharpe":
                        results.loc[ticker, timeframe] = calculate_sharpe_ratio(timeframe_returns, daily_risk_free)
                    elif what == "sortino":
                        results.loc[ticker, timeframe] = calculate_sortino_ratio(timeframe_returns, daily_risk_free)
                    elif what == "omega":
                        results.loc[ticker, timeframe] = calculate_omega_ratio(timeframe_returns, threshold)
                    else:
                        raise ValueError(f"Invalid indicator: {what}. Choose 'omega', 'sharpe', or 'sortino'.")
                
                except Exception as e:
                    print(f"Error processing {ticker} for {timeframe} days: {e}")
                    results.loc[ticker, timeframe] = np.nan
                    
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
            # If we can't get data for the ticker at all, mark all timeframes as NaN
            for timeframe in timeframes:
                results.loc[ticker, timeframe] = np.nan
    
    return results

In [9]:
def calculate_sharpe_ratio(returns, risk_free_rate=0, periods_per_year=252):
    """
    Calculate the Sharpe ratio.
    
    Parameters:
    -----------
    returns : pandas.Series
        Series of returns
    risk_free_rate : float
        Daily risk-free rate
    periods_per_year : int
        Number of trading periods in a year (default: 252 for daily returns)
        
    Returns:
    --------
    float
        Sharpe ratio (annualized)
    """
    # Calculate excess returns
    excess_returns = returns - risk_free_rate
    
    # Calculate mean and standard deviation of excess returns
    mean_excess_returns = excess_returns.mean()
    std_excess_returns = excess_returns.std()
    
    # Avoid division by zero
    if std_excess_returns == 0:
        return np.nan
    
    # Calculate Sharpe ratio
    sharpe_ratio = mean_excess_returns / std_excess_returns

    return sharpe_ratio

    # Calculate annualized Sharpe ratio
    # annualized_sharpe = (mean_excess_returns / std_excess_returns) * np.sqrt(periods_per_year)
    
    # return annualized_sharpe

In [10]:
def calculate_sortino_ratio(returns, risk_free_rate=0, periods_per_year=252):
    """
    Calculate the Sortino ratio.
    
    Parameters:
    -----------
    returns : pandas.Series
        Series of returns
    risk_free_rate : float
        Daily risk-free rate
    periods_per_year : int
        Number of trading periods in a year (default: 252 for daily returns)
        
    Returns:
    --------
    float
        Sortino ratio (annualized)
    """
    # Calculate excess returns
    excess_returns = returns - risk_free_rate
    
    # Calculate mean excess returns
    mean_excess_returns = excess_returns.mean()
    
    # Calculate downside deviation
    negative_returns = excess_returns[excess_returns < 0]
    
    # Avoid division by zero or insufficient data
    if len(negative_returns) <= 1:
        return np.nan
    
    downside_deviation = np.sqrt(np.mean(negative_returns ** 2))
    
    # Avoid division by zero
    if downside_deviation == 0:
        return np.nan
    
    # Calculate Sortino ratio
    sortino_ratio = mean_excess_returns / downside_deviation
    
    # Calculate annualized Sortino ratio
    # annualized_sortino = (mean_excess_returns / downside_deviation) * np.sqrt(periods_per_year)
    
    # return annualized_sortino

In [11]:
def calculate_omega_ratio(returns, threshold=0):
    """
    Calculate the Omega ratio.
    
    Parameters:
    -----------
    returns : pandas.Series
        Series of returns
    threshold : float
        Minimum acceptable return (default: 0)
        
    Returns:
    --------
    float
        Omega ratio
    """
    # Returns above and below threshold
    returns_above = returns[returns > threshold] - threshold
    returns_below = threshold - returns[returns < threshold]
    
    # Sum of returns above and below threshold
    sum_above = returns_above.sum()
    sum_below = returns_below.sum()
    
    # Avoid division by zero
    if sum_below == 0:
        return np.inf
    
    return sum_above / sum_below

### Understanding the Indicators

The Omega ratio represents the ratio of the probability-weighted gains to the probability-weighted losses relative to a threshold (usually zero or risk-free rate). It captures all moments of the return distribution, not just the mean and variance.
Formula:
Omega(τ) = ∫τ∞ [1-F(r)]dr / ∫-∞τ F(r)dr
Where:

τ is the threshold return
F(r) is the cumulative distribution function of returns

Interpretation:

Omega ratio > 1: Investment has more probability-weighted gains than losses
Omega ratio < 1: Investment has more probability-weighted losses than gains
Higher omega ratio is better

Sharpe Ratio
The Sharpe ratio measures the excess return per unit of risk. It quantifies how much additional return you're receiving for the additional volatility of holding a riskier asset.
Formula:
Sharpe Ratio = (Rp - Rf) / σp
Where:

Rp is the portfolio return
Rf is the risk-free rate
σp is the standard deviation of portfolio returns (total risk)

Interpretation:

Sharpe ratio < 1: Poor (return doesn't justify the risk)
Sharpe ratio 1-2: Acceptable to good
Sharpe ratio > 2: Very good
Sharpe ratio > 3: Excellent

Sortino Ratio
The Sortino ratio is similar to the Sharpe ratio but uses downside deviation instead of standard deviation. It focuses only on the harmful volatility by considering only negative returns.
Formula:
Sortino Ratio = (Rp - Rf) / σd
Where:

Rp is the portfolio return
Rf is the risk-free rate
σd is the downside deviation (standard deviation of negative returns only)

Interpretation:

Similar to Sharpe ratio but generally higher as it ignores positive volatility
Typically values > 2 are considered good

In [12]:
# Get Sharpe ratios for default top 20 cryptocurrencies and timeframes using CCXT as data source
sharpe_results = get_indicator(what="sharpe", data_source="ccxt")

# Using CoinGecko as data source
#sharpe_results_cg = get_indicator(what="sharpe", data_source="coingecko")

# Display results
sharpe_results

,2000,1000,365,180,90,30,15,7
BTC,0.054753,0.081777,0.071291,0.040366,0.161481,0.313013,0.329046,-0.369529
ETH,0.056631,0.044691,0.02339,0.039957,0.238293,0.551735,0.976303,1.308083
SOL,0.066992,0.06195,0.025906,-0.008293,0.110237,0.388351,0.625477,0.90042
BNB,0.062851,0.049407,0.03822,0.03363,0.149489,0.482739,0.803157,1.285878
XRP,0.050482,0.064403,0.119269,0.035351,0.177184,0.518853,0.74761,0.48691
DOGE,0.050538,0.053231,0.059703,-0.007223,0.117711,0.458491,0.710076,0.931915
ADA,0.051666,0.038514,0.057389,0.020821,0.098535,0.45569,0.963718,1.085352
AVAX,0.044811,0.032356,0.010676,-0.017574,0.05748,0.398166,0.813374,0.944635
SHIB,0.022177,0.031122,0.012587,-0.015385,0.05564,0.381451,0.700973,1.140044
DOT,0.030534,0.010626,0.000269,-0.0297,0.051902,0.307564,0.804126,1.05165


In [13]:
# Get Sharpe ratios for default top 20 cryptocurrencies and timeframes using CCXT as data source
sortino_results = get_indicator(what="sortino", data_source="ccxt")

# Using CoinGecko as data source
#sharpe_results_cg = get_indicator(what="sharpe", data_source="coingecko")

# Display results
sortino_results

,2000,1000,365,180,90,30,15,7
BTC,None,None,None,None,None,None,None,None
ETH,None,None,None,None,None,None,None,NaN
SOL,None,None,None,None,None,None,None,NaN
BNB,None,None,None,None,None,None,None,NaN
XRP,None,None,None,None,None,None,None,None
DOGE,None,None,None,None,None,None,None,NaN
ADA,None,None,None,None,None,None,None,NaN
AVAX,None,None,None,None,None,None,None,NaN
SHIB,None,None,None,None,None,None,None,NaN
DOT,None,None,None,None,None,None,None,NaN


In [14]:
# Get Sharpe ratios for default top 20 cryptocurrencies and timeframes using CCXT as data source
omega_results = get_indicator(what="omega", data_source="ccxt")

# Using CoinGecko as data source
#sharpe_results_cg = get_indicator(what="sharpe", data_source="coingecko")

# Display results
omega_results

,2000,1000,365,180,90,30,15,7
BTC,1.178929,1.270926,1.225297,1.12573,1.577931,2.440708,2.65774,0.352804
ETH,1.181836,1.141732,1.069416,1.126805,2.161789,4.905804,23.775754,210.862016
SOL,1.214495,1.195008,1.074754,0.976485,1.343829,2.943563,7.093687,110.979391
BNB,1.23365,1.157223,1.113004,1.101458,1.5242,3.739258,11.215967,inf
XRP,1.188868,1.255979,1.464444,1.119688,1.649892,5.039549,15.130499,6.87327
DOGE,1.32228,1.173018,1.183793,0.97941,1.421531,3.463413,8.735415,24.223739
ADA,1.168951,1.130425,1.216608,1.085128,1.306267,3.339346,23.522542,27.925996
AVAX,1.140738,1.093043,1.028141,0.954354,1.160218,2.830931,10.830957,11.784856
SHIB,1.078412,1.103819,1.036647,0.958211,1.167808,2.776683,8.395296,24.846774
DOT,1.094861,1.030144,1.000725,0.923704,1.146579,2.275615,11.136899,17.579646
